# dfcleaner: usage examples

## Importing `dfcleaner` in a project:

To use `dfcleaner` in a project we have to import its subpackages. 

In [3]:
# import dfcleaner
# print(dfcleaner.__version__)

from dfcleaner import catvars # for methods operating on categorical variables 
from dfcleaner import strings # for methods operating on strings